# Statistics and exports
In this notebook, we use the results generated by the ```functionalConnectivityRaw.jl``` in jld2 files to calculate a bunch of statistics per pair and export them to a few formats, including ```.js``` files that are underlying the website. This notebook follows the ```code/statsAndExports.jl``` script (exports are commented out as they would be useless in the Binder).

### Preparing the environment
We want to be in ```CX-Functional-Analysis```

In [ ]:
cd("..")

### Load packages

In [ ]:
using JLD2,DataFrames,AxisArrays,FileIO,CSV
using Interpolations
using StatsBase,Distributions
using JSON
using DataStructures
using Distances,Bootstrap
using HTTP

In [ ]:
include("code/functions/fluoRunUtilities.jl")

### Load data
We're loading the labbook (as a data frame), the table containing the drivers info and the dataset of the fluorescent traces. The jld2 files are on the OSF framework storage, so we need to interact with their API to get the download links.

In [ ]:
reqLabbook = HTTP.get("https://api.osf.io/v2/nodes/vsa3z/files/osfstorage/?filter[name]=labbookTable.jld2")
labbookLink = JSON.parse("$(Char.(reqLabbook.body)...)")

reqData = HTTP.get("https://api.osf.io/v2/nodes/vsa3z/files/osfstorage/?filter[name]=rawData.jld2")
rawLink = JSON.parse("$(Char.(reqData.body)...)")

In [ ]:
labbook = load(download(labbookLink["data"][1]["links"]["download"]),"labbook")
linesToType = CSV.read("LinesAndTypes.csv",weakrefstrings=false)
full_data_dict = load(download(rawLink["data"][1]["links"]["download"]))

```full_data_dict``` is a dictionary with one entry per experiment (with keys following the naming convention "jun1417-Fly1-EB", the last part being the region imaged). Each entry is an array with each element corresponding to one experimental run. Each element is a tuple of :
- one AxisArray (the fluorescent traces)
- one dictionary of metadata (the stimulus conditions and the time to drug exposure if relevant)

The AxisArray : 

In [ ]:
full_data_dict["jun1417-Fly1-EB"][6][1]

And the metadata :

In [ ]:
full_data_dict["jun1417-Fly1-EB"][6][2]

#### Processed versions of the results
We're adding one version of the data that is averaged per run (i.e. the repeats are lumped together) :

In [ ]:
avg_data_dict = map(full_data_dict) do full  
    (k,full) = full
    meanSignals = [(AxisArray(vec(mean(f[1].data,3)),axes(f[1],Axis{:time})),f[2]) for 
        f in full]
    Pair(k,meanSignals)    
end;

This can be saved locally (and used in other scripts) : (useless for this Binder as we're loading those files from the OSF usually).

In [ ]:
## save("data/interpolatedData.jld2",large_interpolated_data_dict)

Then a version of the data where the fluorescence is linearly interpolated on a common time grid. This is used to calculate some statistics (for example correlations between flies) or average experiments together. 2 versions are created, one only contains the times around the stimulation (3 to 5 seconds), the other a large chunk of the recordings (1 to 12 seconds) in 100ms increments :

In [ ]:
interpolated_data_dict = map(avg_data_dict) do dct
        (k,val) = dct
        newAvg = [interpolate_run(run,3:0.1:5) for run in val]     ## The two seconds following the stimulation
        Pair(k,newAvg)
end;

large_interpolated_data_dict = map(avg_data_dict) do dct
        (k,val) = dct
        newAvg = [interpolate_run(run,1:0.1:12) for run in val]     ## The two seconds following the stimulation
        Pair(k,newAvg)
end;

Again, this can be saved to be reused by the figures script.

In [ ]:
#save("data/interpolatedData.jld2",large_interpolated_data_dict)

#### Calculate statisitics per repeat
For every single experimental repeat, we calculate a battery of statistics (the function ```compileStats``` is defined in ```code/functions/fluoRunUtilities.jl```). 

In [ ]:
keyEntries = labbook[:keyEntry];
fullStats = compileStats(full_data_dict,keyEntries)
DataFrames.head(fullStats)

#### Statistics per run
We aggregate those statistic per experimental run (each run is constituted by 4 repeats)

In [ ]:
stats_per_run = aggregate(fullStats,[:experiment,:runIdx],median);

And reformat and add some metadata to this table :

In [ ]:
## Convert the number of pulses back to integer
stats_per_run[:nPulses_median] = convert(Array{Int},stats_per_run[:nPulses_median]);

## Add some metadata columns 
stats_per_run[:cellPair]=""
stats_per_run[:genotype]=""
stats_per_run[:preNeuron]=""
stats_per_run[:postNeuron]=""
stats_per_run[:preDrug]=true
stats_per_run[:Drug]=""
stats_per_run[:timeToDrug]= -Inf
for i in 1:size(stats_per_run,1)
    stats_per_run[i,:cellPair] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellToCell]
    stats_per_run[i,:genotype] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:genotypeRegion]
    stats_per_run[i,:preNeuron] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellPre]
    stats_per_run[i,:postNeuron] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellPost]
    if (!ismissing(labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:Drug]))
        stats_per_run[i,:preDrug] = (labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),
            :timesToDrug][stats_per_run[i,:runIdx]])>Dates.Second(0)
        stats_per_run[i,:timeToDrug] = -Int64(Dates.value(labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),
            :timesToDrug][stats_per_run[i,:runIdx]]))/60000
        stats_per_run[i,:Drug]=labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:Drug]
    end
end

We want to restrict our analysis to cell pairs for which we have a sufficient number of experiments :

In [ ]:
## Calculate the number of runs per pair and select pairs with at least 3 runs
ns = by(stats_per_run,:cellPair,df -> DataFrame(n = length(unique(df[:experiment]))))
fullExps = ns[ns[:n].>=3,:cellPair];

stats_per_run = stats_per_run[in.(Array(stats_per_run[:cellPair]),[fullExps]),:];

We also want to categorize the cell pairs in function of their potential synaptic overlaps. For that we use the ```linesAndTypes``` table which has columns for the pre and post-synaptic innervation patterns.

In [ ]:
## List the cell types tested
uniqueTypesUsed = unique(vcat(stats_per_run[:preNeuron],stats_per_run[:postNeuron]))
sort!(uniqueTypesUsed,by= v->linesToType[findfirst(linesToType[Symbol("Type Description")].==v),:Supertype] )

## Parse the neuron types to define their pre/post regions
possibleNeuropiles = ["PB","FB","EB","NO","GA","LAL","rub","BU"];
neuronTypes = OrderedDict(uniqueTypesUsed[i] =>
                          Dict("innervates" => filter(x -> contains(uniqueTypesUsed[i],x),
                                                                possibleNeuropiles), 
                               "pre"=> split(linesToType[findfirst(linesToType[:,
                                            Symbol("Type Description")].==uniqueTypesUsed[i]),
                                            Symbol("Pre regions")],","),
                               "post" => split(linesToType[findfirst(linesToType[:,
                                            Symbol("Type Description")].==uniqueTypesUsed[i]),
                                            Symbol("Post regions")],","),
                               "pre_fine"=> split(linesToType[findfirst(linesToType[:,
                                                Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                Symbol("Pre regions fine")],","),
                                "post_fine" => split(linesToType[findfirst(linesToType[:,
                                                Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                Symbol("Post regions fine")],","),
                                "short_name" => linesToType[findfirst(linesToType[:,
                                                    Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                    Symbol("New Type Name")]) for i in 1:length(uniqueTypesUsed))

## Using the annotation, establish if there's a potential overlap for every pair 
stats_per_run[:overlapping]=
[(length(intersect(neuronTypes[ty]["pre_fine"],
        neuronTypes[tyPost]["post_fine"]))>0) for (ty,tyPost) in zip(stats_per_run[:preNeuron],
                                                                     stats_per_run[:postNeuron])]

## Is the same neuron used for recording and stimulation ?
stats_per_run[:self]= (stats_per_run[:preNeuron].==stats_per_run[:postNeuron]);

stats_per_run[:expType] = "Non overlapping"
stats_per_run[stats_per_run[:overlapping],:expType] = "Overlapping"
stats_per_run[stats_per_run[:self],:expType] = "Self stimulation"
stats_per_run[:overlapping] =  stats_per_run[:expType].=="Overlapping"; ## Excluding self activation

We then scale a couple of variables so that inhibition spans the -1 to 0 range while excitation spans 0 to 1 :

In [ ]:
stats_per_run[:integNorm_scaled] =  scaleResponse(stats_per_run[:integNorm_median],trimming=true)
stats_per_run[:integral_to_peak_scaled] =  scaleResponse(stats_per_run[:integral_to_peak_median],trimming=true);

```stats_per_run``` now looks like :

In [ ]:
DataFrames.head(stats_per_run,3)

#### Drug experiments tables
We then separate the parts of these tables containing the drug experiments per se (starting 5 minutes before the actual application) :

In [ ]:
# Selecting mecamylamine runs    
mecadf = stats_per_run[(stats_per_run[:Drug].=="Mecamylamine") .& (stats_per_run[:timeToDrug].>-5),:]

# Same thing for picrotoxin
picrodf = stats_per_run[(stats_per_run[:Drug].=="Picrotoxin") .& (stats_per_run[:timeToDrug].>-5),:];

## In case we want to save
## save("data/drugTables.jld2","mecadf",mecadf,"picrodf",picrodf,"drugStats",drugStatsDFPerPair)

### Statistics per pairs
We now compile the statistics per cell pair (keeping the different stimulation conditions separate). The ```category_stats``` function used is in ```code/functions/fluoRunUtilities.jl```. We also scale the normalized integral.

In [ ]:
stats_per_pair = by(stats_per_run[stats_per_run[:preDrug],:],[:cellPair,:nPulses_median],category_stats)
stats_per_pair[:integNormScaled] = scaleResponse(stats_per_pair[:integNorm]);
DataFrames.head(stats_per_pair,2)

We then compute the signed distance to the null distribution of responses (as described in the paper) and the significance at 0.01 and 0.005 to the table :

In [ ]:
## Which statistics are included in the distance calculation
stats_to_use = [:integNormScaled,
                 :between_runs_corr
               ];

addDistances!(stats_per_pair,stats_to_use,:integNormScaled)

And add a "global significance" which is the significance at 1% signed by the sign of the response. We also normalize the distance to the maximum response observed at 20 pulses.

In [ ]:
stats_per_pair[:globalSignif] =  sign.(stats_per_pair[:distance]).*stats_per_pair[:signif1]
stats_per_pair[stats_per_pair[:globalSignif].==-0.0,:globalSignif]=0
stats_per_pair[:distanceNorm] = stats_per_pair[:distance]./maximum(trim(abs.(stats_per_pair[stats_per_pair[:nPulses_median].<=20,:distance]),
                                                                        prop=0.01));

We then create a data frame restricted to the 20 pulses stimulations, as it's the one we'll be using mainly for the summaries (figures and website).

In [ ]:
stats_per_pair_20 = stats_per_pair[stats_per_pair[:nPulses_median].==20,:];

We can then save this data for further use (not run) :

In [ ]:
#@save "data/statTables.jld2" stats_per_run stats_per_pair uniqueTypesUsed stats_per_pair_20

## Exports for the website

For exports, we're taking advantage of the JSON package which can translate a Julia dictionary into a json string. For ease of use in the website, we export those json as javascript variables into small .js files (those will be global variables in the website).
- this first function is used to convert the DataFrames into dictionaries :

In [ ]:
function get_dataDict_per_key(pk,data_dict)
    dat = data_dict[pk][1:min(6,end)]
    if length(dat)==0
        return(0)
    else
        dat = Dict(x[2]["pulseNumber"] => transformAxisArray(x[1]) for x in dat)
    end
end

It uses the ```transformAxisArray``` methods defined here, that creates an AxisArray into a dictionary (hence a js variable) directly usable by plotly.js which we use for plotting in the website : 

In [ ]:
function transformAxisArray{T}(aa::AxisArray{T,1})
    Dict("x"=>axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)],"y"=>aa)
end

transformAxisArray{T}(aa::AxisArray{T,3}) = Dict("x"=>axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)],
                                                 "y"=>reshape(aa,(size(aa,1)*size(aa,2),size(aa,3))))

transformAxisArray{T}(AA::Array{AxisArrays.AxisArray{T,1,Array{T,1},
    Tuple{AxisArrays.Axis{:time,StepRangeLen{Float64}}}},1}) =  Dict("x"=>[axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)] for aa in AA],"y"=>AA)

We then format and export the data :

In [ ]:
## Exporting the full fluorescence and the average
out_data = Dict(pk => get_dataDict_per_key(pk,full_data_dict) for pk in keyEntries)
filter!((x,y) -> y!=0,out_data)
    #writeJS("js/full_data.js","FULL_DATA",out_data)

out_data_avg = Dict(pk => get_dataDict_per_key(pk,avg_data_dict) for pk in keyEntries)
filter!((x,y) -> y!=0,out_data_avg)
    #writeJS("js/avg_data.js","AVG_DATA",out_data_avg)

## Exporting a table mapping the experiments to the cell pairs
pairToExp = Dict(cpair => convert(Array{String},labbook[convert(Array{Bool}
                ,labbook[:cellToCell].==cpair),:keyEntry]) for 
        cpair in unique(labbook[:cellToCell]))
#writeJS("js/pairsToExp.js","PAIRS_TO_EXP",pairToExp)

In [ ]:
## Exporting a supertype description table
##Limiting to the experiments that have been done
supertypes = unique(linesToType[:Supertype])

superDict = Dict(s => unique(linesToType[(linesToType[:Supertype].==s) .& 
        [td in uniqueTypesUsed for td in linesToType[Symbol("Type Description")]],Symbol("Type Description")]) for 
    s in supertypes)
superDict = filter((k,x) -> !isempty(x),superDict)

#writeJS("js/supertypes.js","SUPERTYPES",superDict)

Finally the tables :

In [ ]:
superSummary = 
Dict(cp => Dict(string(n) => 
stats_per_pair_20[(stats_per_pair_20[:cellPair].==cp),n][1] for 
            n in [:n,:expType,:signif1,:signif5,:distanceNorm]) for
            cp in unique(stats_per_pair_20[:cellPair]))

#drugSummary = 
##Dict(cp => Dict(string(n) => 
#drugStatsDFPerPair[(drugStatsDFPerPair[:cellPair].==cp),n][1] for 
#                n in names(drugStatsDFPerPair)) for
#            cp in unique(drugStatsDFPerPair[:cellPair]))

## For now we're exporting the stats for the drug free runs
perRunDataDict = 
Dict(exp => Dict(nP => Dict(string(n) => 
unique(stats_per_run[(stats_per_run[:experiment].==exp) .& (stats_per_run[:nPulses_median].==nP) .& (stats_per_run[:preDrug]),
    n]) for 
                n in names(stats_per_run)) for
    nP in stats_per_run[stats_per_run[:experiment].==exp,:nPulses_median]) for 
    exp in unique(stats_per_run[:experiment]))

## Drug exports
mecaDataDict = 
Dict(exp => Dict(string(n) => 
                 unique(mecadf[(mecadf[:experiment].==exp),n]) for 
                 n in names(mecadf)) for 
     exp in unique(mecadf[:experiment]))

picroDataDict = 
Dict(exp =>  Dict(string(n) => 
                  unique(picrodf[(picrodf[:experiment].==exp),n]) for 
                  n in names(picrodf)) for 
     exp in unique(picrodf[:experiment]))

## A table of drivers, used by the website
drivers = Dict(td => convert(Array,
        linesToType[linesToType[Symbol("Type Description")].== td,:Line]) for 
                          td in unique(linesToType[Symbol("Type Description")]));
#writeJS("js/drivers.js","DRIVERS",drivers)

#writeJS("js/perRunData.js","PER_RUN_DATA",perRunDataDict)
#writeJS("js/neurontypes.js","NEURON_TYPES",neuronTypes)
#writeJS("js/summaryData.js","SUMMARY_DATA",summaryDataDict)
#writeJS("js/superSummary.js","SUPER_SUMMARY",superSummary)
#writeJS("js/mecaData.js","MECA_DATA",mecaDataDict)
#writeJS("js/picroData.js","PICRO_DATA",picroDataDict)